### clean data

In [69]:
import pandas as pd
from sklearn.naive_bayes import BernoulliNB
from sklearn.model_selection import train_test_split

In [70]:
ALLdata = pd.read_csv('../Kobe-Project-master/data.csv')
missing_value = ALLdata['shot_made_flag'].isnull()

In [71]:
data = ALLdata.copy() # create a copy of data frame
ALL_Y = data['shot_made_flag'].copy()

In [72]:
print(data.shape)
data.drop('team_name', axis=1, inplace=True) # Always LA Lakers
print(data.shape)
data.drop('team_id', axis=1, inplace=True) # Always one number
print(data.shape)
data.drop('shot_made_flag', axis=1, inplace=True)
print(data.shape)

(30697, 25)
(30697, 24)
(30697, 23)
(30697, 22)


In [73]:
data.head(0)  

,action_type,combined_shot_type,game_event_id,game_id,lat,loc_x,loc_y,lon,minutes_remaining,period,...,seconds_remaining,shot_distance,shot_type,shot_zone_area,shot_zone_basic,shot_zone_range,game_date,matchup,opponent,shot_id


In [74]:
# Remaining time
data['seconds_from_period_end'] = 60 * data['minutes_remaining'] + data['seconds_remaining']

data.drop('minutes_remaining', axis=1, inplace=True)
data.drop('seconds_remaining', axis=1, inplace=True)

In [75]:
data.head(0)  #21

,action_type,combined_shot_type,game_event_id,game_id,lat,loc_x,loc_y,lon,period,playoffs,...,shot_distance,shot_type,shot_zone_area,shot_zone_basic,shot_zone_range,game_date,matchup,opponent,shot_id,seconds_from_period_end


In [76]:
## Matchup - (away/home)
data['away/home'] = data['matchup'].str.contains('vs').astype('int')
data.drop('matchup', axis=1, inplace=True)

In [77]:
# Game date
data['game_date'] = pd.to_datetime(data['game_date'])
data['game_year'] = data['game_date'].dt.year/1000
data['game_month'] = data['game_date'].dt.month/10
data['game_day'] = data['game_date'].dt.day/10
data.drop('game_date', axis=1, inplace=True)

In [78]:
data.head(0)

,action_type,combined_shot_type,game_event_id,game_id,lat,loc_x,loc_y,lon,period,playoffs,...,shot_zone_area,shot_zone_basic,shot_zone_range,opponent,shot_id,seconds_from_period_end,away/home,game_year,game_month,game_day


In [79]:
# Partition interval
data['loc_x'] = pd.cut(data['loc_x'], 50)
data['loc_y'] = pd.cut(data['loc_y'], 50)
data['lat'] = pd.cut(data['lat'],10)
data['lon'] = pd.cut(data['lon'],10)
data['seconds_from_period_end'] = pd.cut(data['seconds_from_period_end'],100)

In [80]:
data_remov0 = data.copy()
data_remov1 = data.drop('game_day',axis = 1, inplace = False)
data_remov2 = data_remov1.drop('game_id',axis = 1, inplace = False)
data_remov3 = data_remov1.drop('shot_zone_basic',axis = 1, inplace = False)

In [81]:
#One Hot Coding
# Get different feature data set
categorial_cols = list(data_remov0 .columns.values)
for col in categorial_cols:
    dummies = pd.get_dummies(data_remov0 [col])
    dummies = dummies.add_prefix("{}_".format(col))
    data_remov0.drop(col, axis=1, inplace=True)
    data_remov0  = data_remov0 .join(dummies)
data_submit = data_remov0[missing_value]

In [82]:
# data removel the game day
categorial_cols = list(data_remov1.columns.values)
for col in categorial_cols:
    dummies = pd.get_dummies(data_remov1[col])
    dummies = dummies.add_prefix("{}_".format(col))
    data_remov1.drop(col, axis=1, inplace=True)
    data_remov1 = data_remov1.join(dummies)
data_submit = data_remov1[missing_value]
print(data_submit.shape)
print(data_remov1.shape)

(5000, 33347)
(30697, 33347)


In [83]:
# data removel the game_day,game_id
categorial_cols = list(data_remov2.columns.values)
for col in categorial_cols:
    dummies = pd.get_dummies(data_remov2[col])
    dummies = dummies.add_prefix("{}_".format(col))
    data_remov2.drop(col, axis=1, inplace=True)
    data_remov2 = data_remov2.join(dummies)
data_submit = data_remov2[missing_value]
print(data_submit.shape)
print(data_remov2.shape)

(5000, 31788)
(30697, 31788)


In [84]:
# data removel the game_day,game_id and shot_zone_basic
categorial_cols = list(data_remov3.columns.values)
for col in categorial_cols:
    dummies = pd.get_dummies(data_remov3[col])
    dummies = dummies.add_prefix("{}_".format(col))
    data_remov3.drop(col, axis=1, inplace=True)
    data_remov3 = data_remov3.join(dummies)
data_submit = data_remov3[missing_value]
print(data_submit.shape)
print(data_remov3.shape)

(5000, 33340)
(30697, 33340)


### divide dataset

In [85]:
data_remov1.shape

(30697, 33347)

In [86]:
# Separate dataset for training
X0 = data_remov0[~missing_value]
Y = ALL_Y[~missing_value]
X_train0, X_test0, y_train0, y_test0 = train_test_split(X0, Y, test_size=0.33, random_state=123)
X1 = data_remov1[~missing_value]
X_train1, X_test1, y_train1, y_test1 = train_test_split(X1, Y, test_size=0.33, random_state=123)
X2 = data_remov2[~missing_value]
X_train2, X_test2, y_train2, y_test2 = train_test_split(X2, Y, test_size=0.33, random_state=123)
X3 = data_remov3[~missing_value]
X_train3, X_test3, y_train3, y_test3 = train_test_split(X3, Y, test_size=0.33, random_state=123)

In [87]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# train model & predict

completed data

In [88]:
#LogisticRegression model
clf_LR = LogisticRegression()
L_mode0 = clf_LR.fit(X_train0, y_train0)
y_pred0 = L_mode0.predict(X_test0)
print("Accuracy of Logistic Regression model 0: %f" % (accuracy_score(y_pred0, y_test0)))

/Users/sea_crabcai_/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Accuracy of Logistic Regression model 0: 0.652517


data with removing game_day col

In [89]:
L_mode1 = clf_LR.fit(X_train1, y_train1)
y_pred1 = L_mode1.predict(X_test1)
print("Accuracy of Logistic Regression 1: %f" % (accuracy_score(y_pred1, y_test1)))

/Users/sea_crabcai_/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Accuracy of Logistic Regression 1: 0.654522


data with removing game_day and game_id cols

In [90]:
L_mode2 = clf_LR.fit(X_train2, y_train2)
y_pred2 = L_mode2.predict(X_test2)
print("Accuracy of Logistic Regression 2: %f" % (accuracy_score(y_pred2, y_test2)))

/Users/sea_crabcai_/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Accuracy of Logistic Regression 2: 0.671737


data with removing game_day, game_id and shot_zone_basic cols

In [91]:
L_mode3 = clf_LR.fit(X_train3, y_train3)
y_pred3 = L_mode3.predict(X_test3)
print("Accuracy of Logistic Regression 3: %f" % (accuracy_score(y_pred3, y_test3)))

/Users/sea_crabcai_/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Accuracy of Logistic Regression 3: 0.653814


### Cross validation : accuracy & F1 score

In [92]:
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import cross_val_score
cv = ShuffleSplit(n_splits=3, test_size=0.33, random_state=123)

 completed data

In [93]:
LR_acc = cross_val_score(clf_LR, X0, Y, cv=cv, scoring='accuracy')
LR_f1 = cross_val_score(clf_LR, X0, Y, cv=cv, scoring='f1')
print("Accuracy of model and its mean are :",LR_acc,LR_acc.mean())
print("f1 score of model and its mean are :" ,LR_f1,LR_f1.mean())

/Users/sea_crabcai_/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/sea_crabcai_/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/sea_crabcai_/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/sea_crabcai_/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/sea_crabcai_/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: 

Accuracy of model and its mean are : [0.65251739 0.64414574 0.6559368 ] 0.6508666430845419
f1 score of model and its mean are : [0.56565954 0.5539462  0.56719074] 0.5622654964307312


data with removing game_day col

In [94]:
LR_acc = cross_val_score(clf_LR, X1, Y, cv=cv, scoring='accuracy')
LR_f1 = cross_val_score(clf_LR, X1, Y, cv=cv, scoring='f1')
print("Accuracy of model and its mean are :",LR_acc,LR_acc.mean())
print("f1 score of model and its mean are :" ,LR_f1,LR_f1.mean())

/Users/sea_crabcai_/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/sea_crabcai_/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/sea_crabcai_/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/sea_crabcai_/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/sea_crabcai_/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: 

Accuracy of model and its mean are : [0.65452187 0.6448532  0.65805919] 0.6524780882757536
f1 score of model and its mean are : [0.56771909 0.55456965 0.56973294] 0.5640072276037782


data with removing game_day and game_id cols

In [96]:
LR_acc = cross_val_score(clf_LR, X2, Y, cv=cv, scoring='accuracy')
LR_f1 = cross_val_score(clf_LR, X2, Y, cv=cv, scoring='f1')
print("Accuracy of model and its mean are :",LR_acc,LR_acc.mean())
print("f1 score of model and its mean are :" ,LR_f1,LR_f1.mean())

/Users/sea_crabcai_/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/sea_crabcai_/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/sea_crabcai_/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/sea_crabcai_/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/sea_crabcai_/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: 

Accuracy of model and its mean are : [0.67173682 0.6604174  0.66619502] 0.6661164170891797
f1 score of model and its mean are : [0.57496183 0.55473098 0.56277992] 0.5641575793811345


data with removing game_day, game_id and shot_zone_basic cols

In [97]:
LR_acc = cross_val_score(clf_LR, X3, Y, cv=cv, scoring='accuracy')
LR_f1 = cross_val_score(clf_LR, X3, Y, cv=cv, scoring='f1')
print("Accuracy of model and its mean are :",LR_acc,LR_acc.mean())
print("f1 score of model and its mean are :",LR_f1,LR_f1.mean())

/Users/sea_crabcai_/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/sea_crabcai_/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/sea_crabcai_/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/sea_crabcai_/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/sea_crabcai_/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: 

Accuracy of model and its mean are : [0.65381441 0.64497111 0.65841292] 0.6523994811932555
f1 score of model and its mean are : [0.56721698 0.55385983 0.57011426] 0.5637303574980558
